In [2]:
# DELIVERABLE FOR THE PROJECT:
# "KINEMATICS OF MOBILITY"
# CIMMYT 
# David Pastor-Escuredo (Life D Lab)
# Licencia MIT

In [3]:
#Copyright <2019> <David Pastor Escuredo>

#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import pandas as pd
import numpy as np
import json
from nltk.tokenize import word_tokenize
import re
import networkx as nx
import matplotlib.pyplot as plt
import datetime
import collections
from sklearn.cluster import KMeans
from PIL import Image

def getday(d):
    if d<10:
        ds='0'+str(d)
    else:
        ds=str(d)
    return ds

In [3]:
ant_file='/Volumes/OS Disk/Tef_Colombia/datos/CONSULTA_SECTORES.xls'
ant_file_mun='antennas/CONSULTA_SECTORES.xls'
ant_file='antennas/antennas.csv'
xls=pd.ExcelFile(ant_file_mun)#,decimal=',')
sheetMun=xls.parse(0)
print(len(sheetMun))
mun=sheetMun['COD_DANE_MUNICIPIO']

sheet=pd.read_csv(ant_file,delimiter=';')
print(len(sheet))
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
lon=sheet['LONGITUD']
lat=sheet['LATITUD']
sheet['antenna_id']=LAC+Cell
sheet['municipio']=mun
#print(sheet['antenna_id'])
aid=sheet['antenna_id']
print(len(sheet))

29869
29869
   LATITUD  LONGITUD Celda_HEX LAC_HEX
0   4.6092  -74.0718      2711    083F
1   4.6092  -74.0718      2712    083F
2   4.6092  -74.0718      2713    083F
3   4.6092  -74.0718      2714    083F
4   4.6092  -74.0718      2715    083F
29869


In [4]:
mun_data=sheet[sheet['municipio']=='BOGOTÁ D.C.']
print(len(mun_data))
sheet=mun_data
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
lon=sheet['LONGITUD']
lat=sheet['LATITUD']
aid=sheet['antenna_id']

sheet.to_csv('antennas/antennas_bogota.csv')

6757


In [ ]:
mun_data=sheet[sheet['municipio']=='BOGOTÁ D.C.']
print(len(mun_data))
sheet=mun_data
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
lon=sheet['LONGITUD']
lat=sheet['LATITUD']
aid=sheet['antenna_id']

sheet.to_csv('antennas/antennas_bogota.csv')

In [11]:
#lat=pd.to_numeric(lat)
#lon=pd.to_numeric(lon)
minlat=np.float64(np.min(lat))
maxlat=np.float64(np.max(lat))
minlon=np.float64(np.min(lon))
maxlon=np.float64(np.max(lon))
print(minlat,maxlat,minlon,maxlon)

minlon=-83
minlat=0

sheet2=sheet[sheet['LONGITUD']<0]
sheet3=sheet2[sheet2['LATITUD']>0]

print(len(sheet))
print(len(sheet2))
print(len(sheet3))

LAC=sheet3['LAC_HEX']
Cell=sheet3['Celda_HEX']
lon=sheet3['LONGITUD']
lat=sheet3['LATITUD']
aid=sheet3['antenna_id']

minlat=np.float64(np.min(lat))
maxlat=np.float64(np.max(lat))
minlon=np.float64(np.min(lon))
maxlon=np.float64(np.max(lon))
print(minlat,maxlat,minlon,maxlon)

4.47804 4.81472 -74.207 74.0386
6757
6756
6756
4.47804 4.81472 -74.207 -74.0227


In [27]:
cdr_folder='/Volumes/OS Disk/Tef_Colombia/data_sorted/'
year=['2014']
month=['01','02','03','04','05']
day=[31,28,31,30,31]
cd=0
c=0
count2=0

for y in year:
    for m in month:
        for di in range (11, day[cd]+1):
            Im=np.zeros((3000,3000))
            d=getday(di)
            cdr_file=y+m+d
            print(cdr_file)
            with open(cdr_folder+cdr_file, 'r') as fl:
                #D_rows = pd.read_csv(fl, header=None)
                D_rows=json.load(fl)
            print(len(D_rows))
            for row in D_rows:
                #print(D_rows[0])
                my_id=row[1]
                #mlac=D_rows[0][1][0:4]
                #mcell=D_rows[0][1][4:8]
                #print(mlac)
                #print(mcell)
                ix=aid.index[aid==my_id]
                if len(ix)>0:
                    ix=ix[0]
                    #print(ix)
                    llon=lon.loc[ix]
                    llat=lat.loc[ix]
                    if not pd.isnull(llon) and not pd.isnull(llat) and not type(llon) is str and not type(llat) is str:
                        mlon=np.float64(llon)
                        mlat=np.float64(llat)
                        #print(mlon)
                        #print(mlat)
                        ilon=(mlon-minlon)*5000.0
                        ilat=(mlat-minlat)*5000.0
                        if ilon<0:
                            ilon=0
                        if ilat<0:
                            ilat=0
                        
                        ilon=np.uint64(ilon)
                        ilat=np.uint64(ilat)
                        if ilon>2999:
                            ilon=2999
                        if ilat>2999:
                            ilat=2999
                        #print(ilon)
                        #print(ilat)
                        Im[ilat][ilon]=Im[ilat][ilon]+1
                    #print(mlon)
                    #print(mlat)
                else:
                    count2=count2+1
            MIm=Image.fromarray(Im)
            MIm.save('imagesBogota/image_'+str(c)+'.tiff')
            c=c+1
            MIm.show()
            
        cd=cd+1
        print("Non found antennas "+str(count2))

20140111
11847021
20140112
9564113
20140113
13007215
20140114
12708108
20140115
12790514
20140116
12832552
20140117
13387667
20140118
12375479
20140119
9747052
20140120
13138528
20140121
12919871
20140122
12736071
20140123
12648613
20140124
13255080
20140125
12152131
20140126
9631352
20140127
13087719
20140128


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/OS Disk/Tef_Colombia/data_sorted/20140128'

In [22]:
import cv2
import scipy.ndimage as ndimage
dilsize=10
gausssize=10
for c in range(0,17):
    Im=Image.open('imagesBogota/image_'+str(c)+'.tiff')
    #print(np.max(Im))
    mmax=np.max(Im)
    ImN=Im/mmax
    ImN=np.uint8(ImN*255)
    ImN[ImN>0]=255
    a=ImN>0
    sa=np.sum(a[:])
    #print(sa)
    #print(np.max(ImN))
    #print(np.mean(ImN))
    Im2=np.flip(ImN,axis=0)
    Im3=Im2[1100:3000,0:1100]
    MIm3=Image.fromarray(np.uint8(Im3))
    kernel = np.ones((5,5),np.uint8)
    #Im4 = cv2.dilate(Im3,kernel,iterations = 1)
    Im4 = ndimage.morphology.grey_dilation(Im3,size=(dilsize,dilsize))
    Im5 = ndimage.gaussian_filter(Im4, sigma=(gausssize, gausssize), order=0)
    MIm4=Image.fromarray(np.uint8(Im4))
    MIm5=Image.fromarray(np.uint8(Im5))
    MIm3.save('imagesBogota/imagegood_'+str(c)+'.tiff')
    MIm4.save('imagesBogota/imagedilated_'+str(c)+'.tiff')
    MIm5.save('imagesBogota/imagegauss_'+str(c)+'.tiff')
    #MIm4.show()
    #MIm5.show()

#ImN2=np.zeros(np.shape(ImN))
#MIm2=MImN.rotate(180)
#MIm2.save('images/imageReshape_'+str(c)+'.tiff')
#MIm2.show()
#plt.hist(MIm2.getdata())
#plt.show()

11054.0
9338.0
12422.0
12275.0
12677.0
12471.0
12915.0
13173.0
10097.0
12711.0
12620.0
12535.0
12184.0
13289.0
12852.0
9974.0
12975.0


In [23]:
frame=cv2.imread('imagesBogota/imagegauss_'+str(0)+'.tiff')
height, width, layers = frame.shape
video_name='video_gauss_bogota.avi'
video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for c in range(0,17):
    frame=cv2.imread('imagesBogota/imagegauss_'+str(c)+'.tiff')
    video.write(frame)

cv2.destroyAllWindows()
video.release()